In [1]:
%pip install -q -U keras-tuner
%pip install -q -U tensorboard-plugin-profile

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

## Data Fetching

In [3]:
from utils.data_fetch import read_data
data = read_data()

C:\Users\Vardan\AppData\Local\Temp\ipykernel_7084\1164869249.py:2: DeprecationWarning: [Deprecated][in version 6.0.0]: client_factory will be replaced by gspread.http_client types
  data = read_data()


In [4]:
data

,Spindle Speed,Feed (mm/min),Tool Dia. (mm),Wi,Wf,delta Weight,MRR ideal,MRR actual,Delta L,Delta W
0,3000.0,10.0,1.0,49.3753,49.3694,0.0059,3.0,1.076451,0.0480,0.18700
1,3000.0,15.0,1.0,49.3694,49.3682,0.0012,4.5,0.327195,0.0640,0.00200
2,3000.0,20.0,1.0,49.3682,49.3659,0.0023,6.0,0.806629,0.0500,0.14500
3,3250.0,10.0,1.0,49.3659,49.3635,0.0024,3.0,1.007537,0.0565,0.14425
4,3250.0,15.0,1.0,49.3635,49.3617,0.0018,4.5,0.553689,0.0605,0.08075
...,...,...,...,...,...,...,...,...,...,...
103,4750.0,15.0,2.0,47.2419,47.2106,0.0313,15.0,10.826819,0.1850,0.08100
104,4750.0,20.0,2.0,47.2106,47.1822,0.0284,20.0,12.085559,0.1660,0.06300
105,5000.0,10.0,2.0,47.1822,47.1535,0.0287,10.0,6.269954,0.1390,0.07600
106,5000.0,15.0,2.0,47.1535,47.1256,0.0279,15.0,9.038995,0.1390,0.12500


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Spindle Speed   108 non-null    float64
 1   Feed (mm/min)   108 non-null    float64
 2   Tool Dia. (mm)  108 non-null    float64
 3   Wi              108 non-null    float64
 4   Wf              108 non-null    float64
 5   delta Weight    108 non-null    float64
 6   MRR ideal       108 non-null    float64
 7   MRR actual      108 non-null    float64
 8   Delta L         108 non-null    float64
 9   Delta W         108 non-null    float64
dtypes: float64(10)
memory usage: 8.6 KB


## Data Preprocessing

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
train_set, test_set = train_test_split(data, test_size=0.2, random_state=32)

In [7]:
X_train_full = train_set.drop(["Spindle Speed", "Feed (mm/min)"], axis = 1)
y_train_full =pd.concat([train_set["Spindle Speed"], train_set["Feed (mm/min)"]], axis=1)

In [8]:
train_set, val_set = train_test_split(train_set, test_size=0.2, random_state=32)

In [9]:
X_train = train_set.drop(["Spindle Speed", "Feed (mm/min)"], axis = 1)
y_train=pd.concat([train_set["Spindle Speed"], train_set["Feed (mm/min)"]], axis=1)

In [10]:
X_val = val_set.drop(["Spindle Speed", "Feed (mm/min)"], axis = 1)
y_val=pd.concat([val_set["Spindle Speed"], val_set["Feed (mm/min)"]], axis=1)

In [11]:
X_test = test_set.drop(["Spindle Speed", "Feed (mm/min)"], axis = 1)
y_test=pd.concat([test_set["Spindle Speed"], test_set["Feed (mm/min)"]], axis=1)

In [12]:
X_train.shape

(68, 8)

In [13]:
norm_layer = keras.layers.Normalization(input_shape=X_train.shape[1:])

In [14]:
model = keras.models.Sequential([
    norm_layer,
    keras.layers.BatchNormalization(),
    keras.layers.Dense(500, activation="relu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"),
    keras.layers.Dense(2)
])
model.compile(loss="mse", optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics=["RootMeanSquaredError"])
norm_layer.adapt(X_train)

In [15]:
from pathlib import Path
from time import strftime
def get_run_logdir(root_logdir="my_logs"):
 return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")
run_logdir = get_run_logdir() 

In [16]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True)

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 8)                 17        
 on)                                                             
                                                                 
 batch_normalization (Batch  (None, 8)                 32        
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 500)               4500      
                                                                 
 batch_normalization_1 (Bat  (None, 500)               2000      
 chNormalization)                                                
                                                                 
 dense_1 (Dense)             (None, 300)               150300    
                                                        

In [18]:
history = model.fit(X_train,y_train, epochs=500, verbose=1, callbacks=[ early_stopping_cb],validation_data=(X_val,y_val))

Epoch 1/500


3/3 [==============================] - 3s 247ms/step - loss: 8150522.5000 - root_mean_squared_error: 2854.9121 - val_loss: 7879586.0000 - val_root_mean_squared_error: 2807.0601
Epoch 2/500
3/3 [==============================] - 0s 124ms/step - loss: 8138651.5000 - root_mean_squared_error: 2852.8323 - val_loss: 7873729.0000 - val_root_mean_squared_error: 2806.0166
Epoch 3/500
3/3 [==============================] - 0s 69ms/step - loss: 8130603.5000 - root_mean_squared_error: 2851.4211 - val_loss: 7867499.5000 - val_root_mean_squared_error: 2804.9062
Epoch 4/500
3/3 [==============================] - 0s 56ms/step - loss: 8123865.5000 - root_mean_squared_error: 2850.2395 - val_loss: 7860330.5000 - val_root_mean_squared_error: 2803.6282
Epoch 5/500
3/3 [==============================] - 0s 36ms/step - loss: 8116621.0000 - root_mean_squared_error: 2848.9685 - val_loss: 7852420.5000 - val_root_mean_squared_error: 2802.2170
Epoch 6/500
3/3 [==============================] - 0s 55

In [19]:
mse_score, rmse_score = model.evaluate(X_test,y_test)

1/1 [==============================] - 0s 63ms/step - loss: 173413.2656 - root_mean_squared_error: 416.4292


In [20]:
X_new = X_train[:3]

In [21]:
X_train[:3]

,Tool Dia. (mm),Wi,Wf,delta Weight,MRR ideal,MRR actual,Delta L,Delta W
63,3.0,48.3988,48.3622,0.0366,9.0,7.071112,0.328,0.112
31,2.0,49.1654,49.1434,0.0220,9.0,6.014213,0.089,0.234
71,3.0,48.1323,48.1152,0.0171,18.0,6.727256,0.156,0.179


In [22]:
model.predict(X_new)

1/1 [==============================] - 0s 248ms/step


array([[4514.8735  ,   21.604824],
       [3129.0356  ,   14.216042],
       [4283.1914  ,   20.74729 ]], dtype=float32)

In [23]:
y_train[:3]

,Spindle Speed,Feed (mm/min)
63,3750.0,10.0
31,3250.0,15.0
71,4250.0,20.0


### Model Tuning

In [24]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=8, default=2)
    n_neurons = hp.Int("n_neurons", min_value=100, max_value=500)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,
    sampling="log")
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model = keras.Sequential()
    model.add(norm_layer)
    for _ in range(n_hidden):
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.Dense(n_neurons, activation="relu", kernel_initializer="he_normal"))
    model.add(keras.layers.Dense(2))
    model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])
    return model

In [25]:
tuner = kt.RandomSearch(
 build_model, objective=kt.Objective("val_root_mean_squared_error", direction="min"), max_trials=10, overwrite=True,
 directory="Model_Tuning", project_name="my_rnd_search", seed=42)

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)

tuner.search(X_train, y_train, epochs=150, validation_data=(X_val, y_val), callbacks=[early_stop])

Trial 10 Complete [00h 01m 19s]
val_root_mean_squared_error: 284.13201904296875

Best val_root_mean_squared_error So Far: 117.7040786743164
Total elapsed time: 00h 08m 43s


In [26]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [27]:
best_trial = tuner.oracle.get_best_trials(num_trials=1)[0]

In [28]:
best_trial.summary()

Trial 03 summary
Hyperparameters:
n_hidden: 2
n_neurons: 492
learning_rate: 0.001715074355925934
Score: 117.7040786743164


In [29]:
Final_model = tuner.get_best_models(num_models=1)[0]

In [30]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir, profile_batch=(100, 200))

In [46]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)

In [47]:
Final_model.fit(X_train_full, y_train_full, epochs=200, callbacks=[tensorboard_cb, early_stopping_cb])

Epoch 1/200
3/3 [==============================] - 0s 17ms/step - loss: 11794.4502 - root_mean_squared_error: 108.6023
Epoch 2/200
3/3 [==============================] - 0s 25ms/step - loss: 19385.7695 - root_mean_squared_error: 139.2328
Epoch 3/200
3/3 [==============================] - 0s 25ms/step - loss: 19763.7559 - root_mean_squared_error: 140.5836
Epoch 4/200
3/3 [==============================] - 0s 23ms/step - loss: 16679.9219 - root_mean_squared_error: 129.1508
Epoch 5/200
3/3 [==============================] - 0s 58ms/step - loss: 10900.4355 - root_mean_squared_error: 104.4052
Epoch 6/200
3/3 [==============================] - 0s 33ms/step - loss: 5953.2651 - root_mean_squared_error: 77.1574
Epoch 7/200
3/3 [==============================] - 0s 35ms/step - loss: 26906.1367 - root_mean_squared_error: 164.0309
Epoch 8/200
3/3 [==============================] - 0s 40ms/step - loss: 10452.6201 - root_mean_squared_error: 102.2381
Epoch 9/200
3/3 [==============================] -

In [48]:
%load_ext tensorboard
%tensorboard --logdir=./my_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 9016), started 3:27:43 ago. (Use '!kill 9016' to kill it.)

In [49]:
X_new

,Tool Dia. (mm),Wi,Wf,delta Weight,MRR ideal,MRR actual,Delta L,Delta W
63,3.0,48.3988,48.3622,0.0366,9.0,7.071112,0.328,0.112
31,2.0,49.1654,49.1434,0.0220,9.0,6.014213,0.089,0.234
71,3.0,48.1323,48.1152,0.0171,18.0,6.727256,0.156,0.179


In [50]:
Final_model.predict(X_new)

1/1 [==============================] - 0s 72ms/step


array([[3736.5122  ,    9.141345],
       [3231.1167  ,   14.416283],
       [4309.2847  ,   19.588493]], dtype=float32)

In [51]:
y_train[:3]

,Spindle Speed,Feed (mm/min)
63,3750.0,10.0
31,3250.0,15.0
71,4250.0,20.0


In [52]:
Final_model.save("Final_neural_network.h5")

e:\Projects\Micromilling_process_optimisation\.venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
